In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from Vocab import *
from model import AttentionModel

import pandas as pd
import os
import string
import random

In [2]:
df = pd.read_csv('stsds.csv')

In [3]:
df = df.sample(frac=1)
df.head()

,sentence_A,sentence_B,relatedness_score
1981,A man and a woman are running together and hol...,A couple is running towards the ocean,0.720
4280,Two men are standing in deep water,Two men are walking through the water,0.680
3829,The person is not going into the water,The man is going into the water,0.700
2281,There is no chef preparing a meal,A chef is preparing some food,0.797
530,A woman is talking on a telephonic device,A woman is talking on a telephone,0.940


In [23]:
#Hyperparams
lr = 1
gamma = 0.95
embed_size = 128
hidden_size = 256
num_epochs = 20

In [24]:
textcat = open('stsds-cat.txt').read()

In [25]:
vocab = Vocabulary(textcat)

In [26]:
vocab_size = vocab.size()

In [27]:
model = AttentionModel(embed_size, hidden_size, vocab_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [34]:
for e in range(num_epochs):
    total_loss = 0
    for i in range(len(df)):
        optimizer.zero_grad()
        
        t_a = torch.tensor(vocab.getSentenceArray(df["sentence_A"][i]))
        t_b = torch.tensor(vocab.getSentenceArray(df["sentence_B"][i]))
        
        out = model(t_a, t_b)
        loss = criterion(out, torch.tensor(df["relatedness_score"][i]).unsqueeze(0).unsqueeze(0))
        loss.backward()
        total_loss += loss.item()
        
        optimizer.step()

    for param_group in optimizer.param_groups:
        param_group['lr'] *= gamma

    print("Epoch", e, "Loss", total_loss)

/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  corr_attn_params = F.softmax(self.lin_attn(corr).view(1, -1))
/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_attn_params = F.softmax(self.lin_attn(test).view(1, -1))
/home/krypt/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 Loss 4.290307271978245
Epoch 1 Loss 4.190427012768832
Epoch 2 Loss 4.097757626975038
Epoch 3 Loss 4.038831820547312
Epoch 4 Loss 3.9920566990417248
Epoch 5 Loss 3.9518744675428445
Epoch 6 Loss 3.917220830377244
Epoch 7 Loss 3.8851089674222923
Epoch 8 Loss 3.856243221709984
Epoch 9 Loss 3.830083267139555
Epoch 10 Loss 3.808562656265905
Epoch 11 Loss 3.786220502691191
Epoch 12 Loss 3.7666988999039823
Epoch 13 Loss 3.7502762360457815
Epoch 14 Loss 3.7309642761816226
Epoch 15 Loss 3.7148869238738156
Epoch 16 Loss 3.701334083424066
Epoch 17 Loss 3.687283981395411
Epoch 18 Loss 3.673215916213641
Epoch 19 Loss 3.658720753539078


In [35]:
torch.save(model, "saved_models/stsds4.pt")

In [37]:
mini = 1
for i in range(len(df)):
    if df["relatedness_score"][i] < mini:
        mini = df["relatedness_score"][i]
print(mini)

0.2
